# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [15]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [16]:

vacay_df=pd.read_csv('output_data/clean_city_data.csv').reset_index()
vacay_df.drop(["level_0","Unnamed: 0", "index"], axis=1, inplace=True)
vacay_df.head()

,City,Latitude,Longitude,Max Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Date
0,Isheyevka,54.4167,48.2667,32.00,100,90,15.66,RU,02/05/21 13:03
1,Marang,5.2056,103.2059,78.80,83,20,3.44,MY,02/05/21 13:07
2,Kadūr,13.5533,76.0131,68.34,58,34,2.33,IN,02/05/21 13:07
3,Bambous Virieux,-20.3428,57.7575,80.60,78,40,8.05,MU,02/05/21 13:02
4,Busselton,-33.6500,115.3333,60.01,71,96,1.01,AU,02/05/21 13:03


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [17]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [18]:
# Store 'Lat' and 'Lng' into  locations 
locations = vacay_df[["Latitude", "Longitude"]].astype(float)
humidity=vacay_df["Humidity (%)"].astype(float)
locations

,Latitude,Longitude
0,54.4167,48.2667
1,5.2056,103.2059
2,13.5533,76.0131
3,-20.3428,57.7575
4,-33.6500,115.3333
...,...,...
573,59.6425,-151.5483
574,60.3725,93.0408
575,70.3705,31.1107
576,55.0607,15.1306


In [19]:
#Plot Heatmap
fig = gmaps.figure(map_type="TERRAIN",center=(2.0, -40.0), zoom_level=3)

In [20]:
#Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False,max_intensity=100,point_radius=1)

In [21]:
#add layer
fig.add_layer(heat_layer)

In [22]:
#Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [46]:
ideal_weather_df=vacay_df[(vacay_df["Max Temperature (F)"]<=84) & 
                        (vacay_df["Max Temperature (F)"]>=72) & 
                        (vacay_df["Wind Speed (mph)"]<10) & 
                        (vacay_df["Cloudiness (%)"]<=10)].reset_index()
ideal_weather_df.head()

,index,City,Latitude,Longitude,Max Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Date
0,29,Port Blair,11.6667,92.7500,78.66,79,9,8.05,IN,02/05/21 13:07
1,94,Pisco,-13.7000,-76.2167,73.40,78,0,9.22,PE,02/05/21 13:01
2,121,Cabo San Lucas,22.8909,-109.9124,82.99,36,1,4.61,MX,02/05/21 13:02
3,131,Hermanus,-34.4187,19.2345,73.00,76,1,1.01,ZA,02/05/21 13:02
4,147,Kruisfontein,-34.0033,24.7314,73.99,77,9,4.00,ZA,02/05/21 13:02


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [47]:
#hotel_name=[]
#hotel_df=pd.DataFrame({"Hotel Name":hotel_name})
ideal_weather_df["Hotel Name"]=""
hotel_df=ideal_weather_df
hotel_df.head()

,index,City,Latitude,Longitude,Max Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Date,Hotel Name
0,29,Port Blair,11.6667,92.7500,78.66,79,9,8.05,IN,02/05/21 13:07,
1,94,Pisco,-13.7000,-76.2167,73.40,78,0,9.22,PE,02/05/21 13:01,
2,121,Cabo San Lucas,22.8909,-109.9124,82.99,36,1,4.61,MX,02/05/21 13:02,
3,131,Hermanus,-34.4187,19.2345,73.00,76,1,1.01,ZA,02/05/21 13:02,
4,147,Kruisfontein,-34.0033,24.7314,73.99,77,9,4.00,ZA,02/05/21 13:02,


In [54]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "location": "22.2855,114.1577",  # philadelphia coords
    "radius":5000,
    "type": "lodging",
    "key": g_key,
}

for index, row in hotel_df.iterrows():
    lati=hotel_df.loc[index,"Latitude"]
    longi=hotel_df.loc[index,"Longitude"]
    lt_lng=f"{lati},{longi}"
    params["location"]=lt_lng
    hotel_data=requests.get(base_url,params=params).json()
    
    try:
        hotel_df.loc[index,"Hotel Name"]=hotel_data['results'][0]['name']
    except IndexError:
        hotel_df.loc[index, "Hotel Name"]="NaN"

In [56]:
hotel_df_cpy=hotel_df.loc[hotel_df["Hotel Name"]!="NaN"]
hotel_df_cpy

,index,City,Latitude,Longitude,Max Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Date,Hotel Name
0,29,Port Blair,11.6667,92.7500,78.66,79,9,8.05,IN,02/05/21 13:07,Welcomhotel Bay Island Port Blair
1,94,Pisco,-13.7000,-76.2167,73.40,78,0,9.22,PE,02/05/21 13:01,Hotel San Isidro Oficial
2,121,Cabo San Lucas,22.8909,-109.9124,82.99,36,1,4.61,MX,02/05/21 13:02,Hotel Tesoro Los Cabos
3,131,Hermanus,-34.4187,19.2345,73.00,76,1,1.01,ZA,02/05/21 13:02,Misty Waves Boutique Hotel
4,147,Kruisfontein,-34.0033,24.7314,73.99,77,9,4.00,ZA,02/05/21 13:02,Oyster Bay House Rental
5,187,Coahuayana Viejo,18.7333,-103.6833,77.00,65,1,6.13,MX,02/05/21 13:04,Hotel Los Arcos
6,221,Castro,-24.7911,-50.0119,75.51,52,0,6.78,BR,02/05/21 13:06,CHACARA BAILLY
7,239,Savannah Bight,16.4500,-85.8500,80.60,69,1,2.30,HN,02/05/21 13:07,Villa on Dunbar Rock
8,259,Celestún,20.8667,-90.4000,80.20,58,0,5.03,MX,02/05/21 13:10,Posada Lilia
9,310,Bang Rakam,16.7585,100.1174,84.00,43,0,5.66,TH,02/05/21 13:10,รถยนDmax สีขาว


In [57]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df_cpy.iterrows()]
hotel_locations = hotel_df_cpy[["Latitude", "Longitude"]]

In [58]:
# Add marker layer ontop of heat map
markers=gmaps.marker_layer(hotel_locations,info_box_content=hotel_info)

# Display figure
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))